In [1]:
import re
import math
import random
import json
from neo4j import GraphDatabase

# Neo4j 연결 (네 환경에 맞게 수정)
URI = "bolt://localhost:7687"
USER = "neo4j"
PASSWORD = "password"

driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))


In [2]:
from new_extractor_model import extract_keywords


/home/alpaco/anaconda3/envs/fridge/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading tokenizer...
Loading model (this can take a while)...


Loading checkpoint shards: 100%|██████████| 8/8 [01:05<00:00,  8.25s/it]


In [17]:
def normalize_basic(text: str) -> str:
    """공백/특수문자 제거 + 소문자. 한글/영문/숫자만 남김."""
    if not isinstance(text, str):
        text = str(text)
    t = re.sub(r"[^0-9A-Za-z가-힣]", "", text)
    return t.lower()

def ensure_list(x):
    if x is None:
        return []
    if isinstance(x, str):
        if not x.strip():
            return []
        return [x]
    return list(x)


def canonicalize_ingredient_list(lst):
    """
    재료 캐노니컬라이징 
    """
    out = []
    for s in lst:
        if not s:
            continue
        s = s.strip().lower()
        if not s:
            continue
        out.append(s)
    # 중복 제거 + 순서 유지
    seen = set()
    uniq = []
    for x in out:
        if x not in seen:
            seen.add(x)
            uniq.append(x)
    return uniq


def softmax(scores, temperature: float = 1.0):
    """온도 조절 가능한 softmax"""
    if not scores:
        return []
    max_s = max(scores)
    exps = [math.exp((s - max_s) / temperature) for s in scores]
    Z = sum(exps)
    if Z == 0:
        return [1.0 / len(scores)] * len(scores)
    return [e / Z for e in exps]

def get_all_user_keywords(kw):
    """
    extract_keywords() 결과(JSON)에서 사용자의 의미 있는 모든 키워드를
    하나의 리스트(flat)로 정리해주는 함수.
    """

    fields = [
        "must_ingredients",
        "optional_ingredients",
        "dish_type",
        "method",
        "situation",
        "health_tags",
        "weather_tags",
        "menu_style",
        "extra_keywords",
    ]

    result = []
    seen = set()

    for f in fields:
        vals = kw.get(f, [])
        for v in vals:
            if v not in seen:
                seen.add(v)
                result.append(v)

    return result



In [24]:
def build_cypher_from_keywords_relaxed(kw: dict, limit: int = 50):
    """
    리스트 정규화 + 중복 제거만 수행
    """

    kw = dict(kw)  # 원본 보호

    def unique_preserve_order(lst):
        seen = set()
        out = []
        for x in lst:
            if x not in seen:
                seen.add(x)
                out.append(x)
        return out

    # --- 1) 리스트 정규화 ---
    kw["dish_type"]          = ensure_list(kw.get("dish_type"))
    kw["method"]             = ensure_list(kw.get("method"))
    kw["situation"]          = ensure_list(kw.get("situation"))
    kw["must_ingredients"]   = ensure_list(kw.get("must_ingredients"))
    kw["optional_ingredients"] = ensure_list(kw.get("optional_ingredients"))
    kw["exclude_ingredients"]  = ensure_list(kw.get("exclude_ingredients"))
    kw["health_tags"]        = ensure_list(kw.get("health_tags"))
    kw["weather_tags"]       = ensure_list(kw.get("weather_tags"))
    kw["menu_style"]         = ensure_list(kw.get("menu_style"))
    kw["extra_keywords"]     = ensure_list(kw.get("extra_keywords"))
    kw["positive_tags"]      = ensure_list(kw.get("positive_tags"))

    # --- 2) 중복 제거 ---
    kw["health_tags"]    = unique_preserve_order(kw["health_tags"])
    kw["extra_keywords"] = unique_preserve_order(kw["extra_keywords"])
    kw["dish_type"]      = unique_preserve_order(kw["dish_type"])
    kw["method"]         = unique_preserve_order(kw["method"])
    kw["situation"]      = unique_preserve_order(kw["situation"])
    kw["menu_style"]     = unique_preserve_order(kw["menu_style"])

    # --- 3) ingredient canonicalization ---
    for key in ["must_ingredients", "optional_ingredients", "exclude_ingredients"]:
        if kw[key]:
            kw[key] = canonicalize_ingredient_list(kw[key])
        else:
            kw[key] = []

    # --- 4) 파라미터 구성 ---
    params = {
        "must_ings": kw["must_ingredients"],
        "opt_ings": kw["optional_ingredients"],
        "exclude_ings": kw["exclude_ingredients"],
        "dish_type": kw["dish_type"],
        "method_list": kw["method"],
        "situation_list": kw["situation"],
        "health_list": kw["health_tags"],
        "weather_list": kw["weather_tags"],
        "menu_style_list": kw["menu_style"],
        "extra_kw_list": kw["extra_keywords"],
        "max_time": kw.get("max_cook_time_min", None),
        "limit_number": limit,
    }

    # 메뉴명 부스트용 리스트 생성 (dish_type + extra_keywords)
    menu_names = kw.get("dish_type", []) + kw.get("extra_keywords", [])
    params["menu_name_list"] = menu_names

    # --- 5) Cypher 생성 (LLM 키워드 매칭만 사용하는 스코어링) ---
    cypher = """
MATCH (r:RecipeV2)
OPTIONAL MATCH (r)-[:HAS_INGREDIENT_V2]->(ing:IngredientV2)
OPTIONAL MATCH (r)-[:IN_CATEGORY_V2]->(cat:CategoryV2)
OPTIONAL MATCH (r)-[:COOKED_BY_V2]->(meth:MethodV2)
OPTIONAL MATCH (r)-[:FOR_SITUATION_V2]->(sit:SituationV2)
OPTIONAL MATCH (r)-[:HAS_HEALTH_TAG]->(h:HealthTag)
OPTIONAL MATCH (r)-[:HAS_WEATHER_TAG]->(w:WeatherTag)
OPTIONAL MATCH (r)-[:HAS_MENU_STYLE]->(ms:MenuStyle)
OPTIONAL MATCH (r)-[:HAS_EXTRA_KEYWORD]->(ek:ExtraKeyword)
WITH r,
     collect(DISTINCT ing.name) AS ingRaw,
     collect(DISTINCT cat.name) AS catRaw,
     collect(DISTINCT meth.name) AS methodRaw,
     collect(DISTINCT sit.name) AS sitRaw,
     collect(DISTINCT h.name) AS healthRaw,
     collect(DISTINCT w.name) AS weatherRaw,
     collect(DISTINCT ms.name) AS menuStyleRaw,
     collect(DISTINCT ek.name) AS extraRaw

// --- 공백 제거 + 소문자 리스트로 정규화 ---
WITH r,
     [x IN ingRaw       | replace(toLower(x), " ", "")] AS ingList,
     [x IN catRaw       | replace(toLower(x), " ", "")] AS catList,
     [x IN methodRaw    | replace(toLower(x), " ", "")] AS methodList,
     [x IN sitRaw       | replace(toLower(x), " ", "")] AS sitList,
     [x IN healthRaw    | replace(toLower(x), " ", "")] AS healthList,
     [x IN weatherRaw   | replace(toLower(x), " ", "")] AS weatherList,
     [x IN menuStyleRaw | replace(toLower(x), " ", "")] AS menuStyleList,
     [x IN extraRaw     | replace(toLower(x), " ", "")] AS extraList

// ----------- HARD FILTERS (must/exclude 재료, 시간) -------------
WHERE (
    size($must_ings) = 0
    OR ANY(ing IN $must_ings WHERE
            ANY(mi IN ingList WHERE mi CONTAINS replace(toLower(ing), " ", "")))
)
AND (
    size($exclude_ings) = 0
    OR NONE(ex IN $exclude_ings WHERE
            ANY(mi IN ingList WHERE mi CONTAINS replace(toLower(ex), " ", "")))
)
AND (
    $max_time IS NULL
    OR r.time_min <= $max_time
)

// ----------- SCORING (LLM 키워드 매칭만 사용) -------------
WITH
    r,
    ingList, catList, methodList, sitList, healthList, weatherList, menuStyleList, extraList,

    // 1) must ingredients
    size([
        ing IN $must_ings
        WHERE ANY(mi IN ingList
                  WHERE mi CONTAINS replace(toLower(ing), " ", ""))
    ]) * 5 AS score_must_ing,

    // 2) optional ingredients
    size([
        ing IN $opt_ings
        WHERE ANY(mi IN ingList
                  WHERE mi CONTAINS replace(toLower(ing), " ", ""))
    ]) * 2 AS score_opt_ing,

    // 3) dish_type (CategoryV2)
    size([
        dt IN $dish_type
        WHERE ANY(cat IN catList
                  WHERE cat CONTAINS replace(toLower(dt), " ", ""))
    ]) * 3 AS score_dish_type,

    // 4) method (MethodV2)
    size([
        mt IN $method_list
        WHERE ANY(m IN methodList
                  WHERE m CONTAINS replace(toLower(mt), " ", ""))
    ]) * 2 AS score_method,

    // 5) situation (SituationV2)
    size([
        st IN $situation_list
        WHERE ANY(s IN sitList
                  WHERE s CONTAINS replace(toLower(st), " ", ""))
    ]) * 4 AS score_situation,

    // 6) health_tags (HealthTag)
    size([
        ht IN $health_list
        WHERE ANY(h IN healthList
                  WHERE h CONTAINS replace(toLower(ht), " ", "")
                     OR replace(toLower(ht), " ", "") CONTAINS h)
    ]) * 5 AS score_health,

    // 7) weather_tags (WeatherTag)
    size([
        wt IN $weather_list
        WHERE ANY(w IN weatherList
                  WHERE w CONTAINS replace(toLower(wt), " ", ""))
    ]) * 3 AS score_weather,

    // 8) menu_style (MenuStyle)
    size([
        ms IN $menu_style_list
        WHERE ANY(m IN menuStyleList
                  WHERE m CONTAINS replace(toLower(ms), " ", ""))
    ]) * 2 AS score_menu_style,

    // 9) extra_keywords (ExtraKeyword)
    size([
        ek IN $extra_kw_list
        WHERE ANY(e IN extraList
                  WHERE e CONTAINS replace(toLower(ek), " ", "")
                     OR replace(toLower(ek), " ", "") CONTAINS e)
    ]) * 3 AS score_extra,

    // 10) menu_name boost (사용자가 특정 메뉴명을 언급한 경우)
    size([
    mn IN $menu_name_list
    WHERE (
        toLower(r.name) CONTAINS replace(toLower(mn), " ", "") OR
        toLower(r.title) CONTAINS replace(toLower(mn), " ", "")
    )
    ]) * 10 AS score_menu_name

WITH
    r,
    score_must_ing,
    score_opt_ing,
    score_dish_type,
    score_method,
    score_situation,
    score_health,
    score_weather,
    score_menu_style,
    score_extra,
    score_menu_name,
    (
        score_must_ing +
        score_opt_ing +
        score_dish_type +
        score_method +
        score_situation +
        score_health +
        score_weather +
        score_menu_style +
        score_extra +
        score_menu_name
    ) AS score

RETURN
    r.recipe_id   AS recipe_id,
    r.title       AS title,
    r.name        AS name,
    r.views       AS views,
    r.time_min    AS time_min,
    r.difficulty  AS difficulty,
    r.servings    AS servings,
    score,
    score_must_ing,
    score_opt_ing,
    score_dish_type,
    score_method,
    score_situation,
    score_health,
    score_weather,
    score_menu_style,
    score_extra,
    score_menu_name
ORDER BY score DESC, r.views DESC
LIMIT $limit_number
    """

    # 정규화된 kw도 함께 반환
    return cypher, params, kw


In [33]:
def _norm_tag(s: str) -> str:
    return str(s).replace(" ", "").lower()


def _build_match_dict(llm_list, graph_list):
    result = {}
    norm_graph = [(g, _norm_tag(g)) for g in graph_list]

    for kw in llm_list:
        norm_kw = _norm_tag(kw)
        if not norm_kw:
            continue
        hits = [
            g for (g, ng) in norm_graph
            if norm_kw in ng or ng in norm_kw
        ]
        if hits:
            result[kw] = hits
    return result


def graph_rag_search_with_scoring_explanation(
    user_prompt: str,
    top_k: int = 5,
    greedy_k: int = 3,          # 점수 그대로 뽑을 개수
    temperature: float = 1.5,   # softmax 온도 (크면 다양성↑)
):
    print("\n" + "=" * 80)
    print("USER PROMPT:", user_prompt)

    # 1) 키워드 추출
    raw_kw = extract_keywords(user_prompt)
    cypher, params, kw = build_cypher_from_keywords_relaxed(raw_kw, limit=50) 

    # 매칭된 키워드 모두 리스트 목록화
    matched_keywords_only = get_all_user_keywords(raw_kw)

    print("=== 사용자가 요청한 의미 키워드 목록 ===")
    print(matched_keywords_only)

    print("\n=== [2] Generated Cypher ===\n")
    print(cypher)
    print("\nParams:", params)

    # 2) Neo4j에서 상위 50개 후보 가져오기
    with driver.session() as session:
        result = session.run(cypher, **params)
        rows = list(result)

    if not rows:
        print("\n⚠️ 조건에 맞는 레시피가 없습니다.")
        return {"keywords": kw, "recipes": []}
    
    
    # 레시피 기반 프롬프트가 주어지지 않는 경우 (극단적이거나 장난스러운 프롬프트 예방)
    all_zero = all((rec["score"] or 0) == 0 for rec in rows)
    if all_zero:
        print("\n⚠️ 점수 기반으로 추천할 만한 레시피가 없습니다. (모든 후보 score=0)")
        return {
            "keywords": kw,
            "recipes": [],
            "no_result_message": "조회 가능한 메뉴가 없습니다. 프롬프트를 조금 더 구체적으로 입력해 주세요.",
        }

    # 최대 50개만 후보로 사용
    top_candidates = rows[:50]

    # 후보 개수가 top_k보다 적으면 그냥 전부 사용
    if len(top_candidates) <= top_k:
        selected_rows = top_candidates
    else:
        # 2-1) 상위 greedy_k개는 점수 순서 그대로
        greedy_k = min(greedy_k, top_k, len(top_candidates))
        greedy_part = top_candidates[:greedy_k]

        # 2-2) 나머지는 softmax로 다양성 있게 뽑기
        diversity_needed = top_k - greedy_k
        diversity_pool = top_candidates[greedy_k:]

        if diversity_needed <= 0 or not diversity_pool:
            selected_rows = greedy_part
        else:
            # softmax 확률 계산 (score 기반)
            scores = [rec["score"] for rec in diversity_pool]
            probs = softmax(scores, temperature=temperature)

            chosen_idx = []
            # 중복 없이 diversity_needed개까지 샘플링
            while len(chosen_idx) < diversity_needed and len(chosen_idx) < len(diversity_pool):
                r = random.random()
                cum = 0.0
                for i, p in enumerate(probs):
                    cum += p
                    if r <= cum:
                        if i not in chosen_idx:
                            chosen_idx.append(i)
                        break

            diverse_part = [diversity_pool[i] for i in chosen_idx]
            # diverse_part 조합 끝난 직후
            selected_rows = greedy_part + diverse_part

            # === 메뉴명 기준 중복 제거 ===
            unique_rows = []
            seen_names = set()

            for rec in selected_rows:
                norm_name = rec["name"].replace(" ", "").lower()
                if norm_name not in seen_names:
                    seen_names.add(norm_name)
                    unique_rows.append(rec)

            if len(unique_rows) < top_k:
                needed = top_k - len(unique_rows)

                # 여기에 중복 아닌 추가 후보 채우기
                for rec in top_candidates:
                    norm_name = rec["name"].replace(" ", "").lower()
                    if norm_name not in seen_names:
                        seen_names.add(norm_name)
                        unique_rows.append(rec)
                        if len(unique_rows) == top_k:
                            break

            # top_k 크기 맞춰주기
            selected_rows = unique_rows[:top_k]

            print(f"\n=== [3] Final {len(selected_rows)} results with scoring explanation (Top-{top_k}) ===\n")


    recipes = []

    # 레시피 태그 디테일 쿼리
    recipe_detail_query = """
    MATCH (r:RecipeV2 {recipe_id: $rid})
    OPTIONAL MATCH (r)-[:HAS_HEALTH_TAG]->(h:HealthTag)
    OPTIONAL MATCH (r)-[:HAS_WEATHER_TAG]->(w:WeatherTag)
    OPTIONAL MATCH (r)-[:HAS_MENU_STYLE]->(ms:MenuStyle)
    OPTIONAL MATCH (r)-[:HAS_EXTRA_KEYWORD]->(ek:ExtraKeyword)
    OPTIONAL MATCH (r)-[:FOR_SITUATION_V2]->(sit:SituationV2)
    OPTIONAL MATCH (r)-[:COOKED_BY_V2]->(meth:MethodV2)
    OPTIONAL MATCH (r)-[:IN_CATEGORY_V2]->(cat:CategoryV2)
    RETURN
      collect(DISTINCT h.name)   AS healthList,
      collect(DISTINCT w.name)   AS weatherList,
      collect(DISTINCT ms.name)  AS menuStyleList,
      collect(DISTINCT ek.name)  AS extraList,
      collect(DISTINCT sit.name) AS situationList,
      collect(DISTINCT meth.name) AS methodList,
      collect(DISTINCT cat.name) AS categoryList
    """

    with driver.session() as session:
        for i, rec in enumerate(selected_rows, start=1):
            r_info = {
                "recipe_id": rec["recipe_id"],
                "title": rec["title"],
                "name": rec["name"],
                "views": rec["views"],
                "time_min": rec["time_min"],
                "difficulty": rec["difficulty"],
                "servings": rec["servings"],
                "score": rec["score"],
                "score_must_ing": rec["score_must_ing"],
                "score_opt_ing": rec["score_opt_ing"],
                "score_dish_type": rec["score_dish_type"],
                "score_method": rec["score_method"],
                "score_situation": rec["score_situation"],
                "score_health": rec["score_health"],
                "score_weather": rec["score_weather"],
                "score_menu_style": rec["score_menu_style"],
                "score_extra": rec["score_extra"],
            }

            detail = session.run(recipe_detail_query, {"rid": rec["recipe_id"]}).single()

            categoryList  = detail["categoryList"]   or []
            methodList    = detail["methodList"]     or []
            situationList = detail["situationList"]  or []
            healthList    = detail["healthList"]     or []
            weatherList   = detail["weatherList"]    or []
            menuStyleList = detail["menuStyleList"]  or []
            extraList     = detail["extraList"]      or []

            expl_lines = []

            # ❶ 매칭 정보 구조 저장용 dict
            matched_tag_dict = {}

            # --- 하드 필터 설명 ---
            if kw.get("must_ingredients"):
                expl_lines.append(
                    f"- 필수 재료(must_ingredients={kw['must_ingredients']}) 모두 포함 → 하드 필터 통과"
                )
            if kw.get("exclude_ingredients"):
                expl_lines.append(
                    f"- 제외 재료(exclude_ingredients={kw['exclude_ingredients']})는 포함되지 않음 → 하드 필터 통과"
                )
            if kw.get("max_cook_time_min"):
                max_t = kw["max_cook_time_min"]
                cur_t = r_info["time_min"]
                if cur_t is not None and cur_t <= max_t:
                    expl_lines.append(
                        f"- 최대 조리시간 {max_t}분 조건 만족 (현재 {cur_t}분)"
                    )
                else:
                    expl_lines.append(
                        f"- 최대 조리시간 {max_t}분 조건 미충족일 수 있음 (time_min={cur_t})"
                    )

            # --- LLM 키워드 기반 매칭 설명 + 매칭 구조 저장 ---

            if kw.get("dish_type"):
                match_dict = _build_match_dict(kw["dish_type"], categoryList)
                matched_tag_dict["dish_type"] = match_dict
                match_cnt = len(match_dict)
                expl_lines.append(
                    f"- [dish_type(CategoryV2)] 점수 {r_info['score_dish_type']}점 (LLM 키워드 매칭 {match_cnt}개)"
                )
                expl_lines.append(f"   · LLM dish_type(CategoryV2) 키워드: {kw['dish_type']}")
                expl_lines.append(f"   · LLM 키워드↔그래프 태그 매칭: {match_dict}")

            if kw.get("method"):
                match_dict = _build_match_dict(kw["method"], methodList)
                matched_tag_dict["method"] = match_dict
                match_cnt = len(match_dict)
                expl_lines.append(
                    f"- [method(MethodV2)] 점수 {r_info['score_method']}점 (LLM 키워드 매칭 {match_cnt}개)"
                )
                expl_lines.append(f"   · LLM method(MethodV2) 키워드: {kw['method']}")
                expl_lines.append(f"   · LLM 키워드↔그래프 태그 매칭: {match_dict}")

            if kw.get("situation"):
                match_dict = _build_match_dict(kw["situation"], situationList)
                matched_tag_dict["situation"] = match_dict
                match_cnt = len(match_dict)
                expl_lines.append(
                    f"- [situation(SituationV2)] 점수 {r_info['score_situation']}점 (LLM 키워드 매칭 {match_cnt}개)"
                )
                expl_lines.append(f"   · LLM situation(SituationV2) 키워드: {kw['situation']}")
                expl_lines.append(f"   · LLM 키워드↔그래프 태그 매칭: {match_dict}")

            if kw.get("health_tags"):
                match_dict = _build_match_dict(kw["health_tags"], healthList)
                matched_tag_dict["health_tags"] = match_dict
                match_cnt = len(match_dict)
                expl_lines.append(
                    f"- [health_tags(HealthTag)] 점수 {r_info['score_health']}점 (LLM 키워드 매칭 {match_cnt}개)"
                )
                expl_lines.append(f"   · LLM health_tags(HealthTag) 키워드: {kw['health_tags']}")
                expl_lines.append(f"   · LLM 키워드↔그래프 태그 매칭: {match_dict}")

            if kw.get("weather_tags"):
                match_dict = _build_match_dict(kw["weather_tags"], weatherList)
                matched_tag_dict["weather_tags"] = match_dict
                match_cnt = len(match_dict)
                expl_lines.append(
                    f"- [weather_tags(WeatherTag)] 점수 {r_info['score_weather']}점 (LLM 키워드 매칭 {match_cnt}개)"
                )
                expl_lines.append(f"   · LLM weather_tags(WeatherTag) 키워드: {kw['weather_tags']}")
                expl_lines.append(f"   · LLM 키워드↔그래프 태그 매칭: {match_dict}")

            if kw.get("menu_style"):
                match_dict = _build_match_dict(kw["menu_style"], menuStyleList)
                matched_tag_dict["menu_style"] = match_dict
                match_cnt = len(match_dict)
                expl_lines.append(
                    f"- [menu_style(MenuStyle)] 점수 {r_info['score_menu_style']}점 (LLM 키워드 매칭 {match_cnt}개)"
                )
                expl_lines.append(f"   · LLM menu_style(MenuStyle) 키워드: {kw['menu_style']}")
                expl_lines.append(f"   · LLM 키워드↔그래프 태그 매칭: {match_dict}")

            if kw.get("extra_keywords"):
                match_dict = _build_match_dict(kw["extra_keywords"], extraList)
                matched_tag_dict["extra_keywords"] = match_dict
                match_cnt = len(match_dict)
                expl_lines.append(
                    f"- [extra_keywords(ExtraKeyword)] 점수 {r_info['score_extra']}점 (LLM 키워드 매칭 {match_cnt}개)"
                )
                expl_lines.append(f"   · LLM extra_keywords(ExtraKeyword) 키워드: {kw['extra_keywords']}")
                expl_lines.append(f"   · LLM 키워드↔그래프 태그 매칭: {match_dict}")

            # === ❷ 사용자가 실제로 요청한 의미 있는 모든 키워드 모아두기 ===

            user_keywords_all = (
                kw.get("must_ingredients", [])
                + kw.get("optional_ingredients", [])
                + kw.get("dish_type", [])
                + kw.get("method", [])
                + kw.get("situation", [])
                + kw.get("health_tags", [])
                + kw.get("weather_tags", [])
                + kw.get("menu_style", [])
                + kw.get("extra_keywords", [])
            )

            # 중복 제거 + 순서 유지
            seen_kw = set()
            flat_unique = []
            for k in user_keywords_all:
                if k not in seen_kw:
                    seen_kw.add(k)
                    flat_unique.append(k)

            # 결과 저장
            r_info["matched_keywords_flat"] = flat_unique


            summary_line = (
                f"총점 {r_info['score']}점 "
                f"(must={r_info['score_must_ing']}, "
                f"opt={r_info['score_opt_ing']}, "
                f"dish={r_info['score_dish_type']}, "
                f"method={r_info['score_method']}, "
                f"situation={r_info['score_situation']}, "
                f"health={r_info['score_health']}, "
                f"weather={r_info['score_weather']}, "
                f"style={r_info['score_menu_style']}, "
                f"extra={r_info['score_extra']})"
            )

            print(f"[{i}] ({r_info['recipe_id']}) {r_info['title']}  | 이름: {r_info['name']}")
            print(f"     - 조리시간: {r_info['time_min']}분 | 난이도: {r_info['difficulty']} | 조회수: {r_info['views']}")
            print("     -", summary_line)
            for line in expl_lines:
                print("        ", line)

            # ❸ r_info에 매칭 정보 저장
            r_info["summary"] = summary_line
            r_info["explanation_lines"] = expl_lines
            r_info["matched_tag_dict"] = matched_tag_dict
            r_info["matched_keywords_flat"] = flat_unique

            recipes.append(r_info)

    return {
        "keywords": kw,
        "recipes": recipes,
    }


In [34]:
user_prompt = "돼지고기와 김치가 들어간 매콤한 요리"

res = graph_rag_search_with_scoring_explanation(
    user_prompt,
    top_k=5
)


USER PROMPT: 돼지고기와 김치가 들어간 매콤한 요리
=== 사용자가 요청한 의미 키워드 목록 ===
['돼지고기', '김치']

=== [2] Generated Cypher ===


MATCH (r:RecipeV2)
OPTIONAL MATCH (r)-[:HAS_INGREDIENT_V2]->(ing:IngredientV2)
OPTIONAL MATCH (r)-[:IN_CATEGORY_V2]->(cat:CategoryV2)
OPTIONAL MATCH (r)-[:COOKED_BY_V2]->(meth:MethodV2)
OPTIONAL MATCH (r)-[:FOR_SITUATION_V2]->(sit:SituationV2)
OPTIONAL MATCH (r)-[:HAS_HEALTH_TAG]->(h:HealthTag)
OPTIONAL MATCH (r)-[:HAS_WEATHER_TAG]->(w:WeatherTag)
OPTIONAL MATCH (r)-[:HAS_MENU_STYLE]->(ms:MenuStyle)
OPTIONAL MATCH (r)-[:HAS_EXTRA_KEYWORD]->(ek:ExtraKeyword)
WITH r,
     collect(DISTINCT ing.name) AS ingRaw,
     collect(DISTINCT cat.name) AS catRaw,
     collect(DISTINCT meth.name) AS methodRaw,
     collect(DISTINCT sit.name) AS sitRaw,
     collect(DISTINCT h.name) AS healthRaw,
     collect(DISTINCT w.name) AS weatherRaw,
     collect(DISTINCT ms.name) AS menuStyleRaw,
     collect(DISTINCT ek.name) AS extraRaw

// --- 공백 제거 + 소문자 리스트로 정규화 ---
WITH r,
     [x IN ingRaw       

In [25]:
from explanation_model import add_llm_explanations

res = add_llm_explanations(user_prompt, res)

# 1) 아예 레시피가 없으면 (점수 0으로 걸러졌거나, 애초에 rows가 없거나)
if not res.get("recipes"):
    msg = res.get(
        "no_result_message",
        "조회 가능한 메뉴가 없습니다. 프롬프트를 다시 입력해 주세요."
    )
    print(msg)

else:
    # 2) 정상적으로 레시피가 있으면 → LLM 추천 이유 붙이기
    res = add_llm_explanations(user_prompt, res)

    for idx, r in enumerate(res["recipes"], start=1):
        expl = r.get("llm_explanation", {})

        print("\n" + "=" * 80)
        print(f"[{idx}] {r['title']} | {r['name']}")
        print(f"  - 점수 요약: {r['summary']}")
        print(f"  - 매칭 키워드(flat): {r.get('matched_keywords_flat', [])}")
        print("  - 추천 이유:", expl.get("short_reason"))


[1] 백종원 돼지갈비 김치찜 만들기 돼지갈비김치찜레시피 묵은지 김치찜 만드는 법 | 돼지갈비김치찜
  - 점수 요약: 총점 10점 (must=10, opt=0, dish=0, method=0, situation=0, health=0, weather=0, style=0, extra=0)
  - 매칭 키워드(flat): ['돼지고기', '김치']
  - 추천 이유: 백종원의 돼지갈비 김치찜 레시피를 추천드립니다. 돼지고기와 김치를 주 재료로 사용하여 원하시는 조합을 충족합니다.

[2] 녹두 빈대떡 만들기 고소한 녹두전 레시피 | 녹두빈대떡
  - 점수 요약: 총점 10점 (must=10, opt=0, dish=0, method=0, situation=0, health=0, weather=0, style=0, extra=0)
  - 매칭 키워드(flat): ['돼지고기', '김치']
  - 추천 이유: 녹두빈대떡은 사용자가 원하는 돼지고기와 김치를 포함하고 있어 추천되었습니다.

[3] 편스토랑 류수영 평생김치찌개 어남선생평생김치찌개 레시피 | 평생김치찌개
  - 점수 요약: 총점 10점 (must=10, opt=0, dish=0, method=0, situation=0, health=0, weather=0, style=0, extra=0)
  - 매칭 키워드(flat): ['돼지고기', '김치']
  - 추천 이유: 평생김치찌개 레시피는 사용자가 원하는 돼지고기와 김치를 주 재료로 사용하여 만들 수 있는 요리입니다.

[4] 돼지고기 김치찜 만들기 집밥 냉파요리 | 돼지고기김치찜
  - 점수 요약: 총점 10점 (must=10, opt=0, dish=0, method=0, situation=0, health=0, weather=0, style=0, extra=0)
  - 매칭 키워드(flat): ['돼지고기', '김치']
  - 추천 이유: 돼지고기와 김치를 주 재료로 사용하여 요청하신 돼지고기 김치찜을 추천드립니다.

[5] 김치찌게 맛있게 끓이기 돼지고기